# Step 1: Scrape the wikipedia for data of the neighborhoods in Toronto
Requirements: 

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [2]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

## 1.1 Scrape from wikipedia using pandas 'read_html'


In [28]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_data = pd.read_html(wiki_url)[0]
print(f"{wiki_data.shape}\n {wiki_data.head()}")


(180, 3)
   Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


## 1.2 cleanup data

In [53]:
toronto_data = wiki_data.loc[wiki_data['Borough'] != "Not assigned"].reset_index(drop=True)
print(f"{toronto_data.shape}\n {toronto_data.head()}")

(103, 3)
   Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


In [54]:
#check if 'M5A' taking one row:
toronto_data[toronto_data['Postal Code']=="M5A"]

,Postal Code,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [57]:
# check if Neighborhood col has valid data, if not, coping over the 'Borough' over:
invalid_str = "Not assigned"
toronto_data.loc[(toronto_data.Neighborhood == invalid_str),'Neighborhood']= toronto_data.Borough
display(toronto_data[toronto_data['Neighborhood']==invalid_str])


,Postal Code,Borough,Neighborhood


## final shape of data:

In [58]:
toronto_data.shape

(103, 3)

# Step 2: get the latitude and the longitude coordinates of each neighborhood.

In [84]:
! pip install pgeocode

In [114]:
from pgeocode import Nominatim

def get_coordinates(postal_code):
  nomi = Nominatim('ca')
  lat_lng_coords = nomi.query_postal_code(postal_code)
  #print(lat_lng_coords.latitude, lat_lng_coords.longitude)

  latitude = lat_lng_coords.latitude
  longitude = lat_lng_coords.longitude
  return (latitude, longitude)

In [115]:
# append coordinates:
coordinate_list = []
for postal, boro, neigh in zip(toronto_data['Postal Code'], toronto_data['Borough'], toronto_data['Neighborhood']):
  
  lat, lon = get_coordinates(postal)
  #print(postal, boro, neigh, lat, lon)
  coordinate_list.append([postal, boro, neigh, lat, lon])

toronto_coord = pd.DataFrame(coordinate_list)
toronto_coord.columns = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [112]:
display(toronto_coord) 


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


# Step 3: Explore and cluster the neighborhoods in Toronto (with only boroughs that contain the word Toronto)


In [160]:
import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

## 3.1 filter data 

In [125]:
neighborhoods = toronto_coord.loc[toronto_coord['Borough'].str.contains('Toronto')]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [143]:
# create map of New York using latitude and longitude values
latitude = neighborhoods["Latitude"][2]
longitude =  neighborhoods["Longitude"][2]
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# 3.2 get venues data from Foursquare

In [163]:
CLIENT_ID = 'FZ4SGISHJHNAUF001B3WARC00KHRISPK2UAKPDHE14CKI1X1' # your Foursquare ID
CLIENT_SECRET = 'DRD55DNPRNKX0DPMKKDAO2QFEO0FRFMHVQERIMS4RA2EDU0S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

toronto_data = neighborhoods.reset_index(drop=True)
display(toronto_data.head())

Your credentails:
CLIENT_ID: FZ4SGISHJHNAUF001B3WARC00KHRISPK2UAKPDHE14CKI1X1
CLIENT_SECRET:DRD55DNPRNKX0DPMKKDAO2QFEO0FRFMHVQERIMS4RA2EDU0S


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


In [165]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

search_query = neighborhood_name
latitude = neighborhood_latitude
longitude = neighborhood_longitude
LIMIT = 100
radius = 500

print(search_query + ' .... OK!')

#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print(url)
results = requests.get(url).json()
results.keys()


Latitude and longitude values of Regent Park, Harbourfront are 43.6555, -79.3626.
Regent Park, Harbourfront .... OK!
https://api.foursquare.com/v2/venues/explore?client_id=FZ4SGISHJHNAUF001B3WARC00KHRISPK2UAKPDHE14CKI1X1&client_secret=DRD55DNPRNKX0DPMKKDAO2QFEO0FRFMHVQERIMS4RA2EDU0S&ll=43.6555,-79.3626&v=20180605&radius=500&limit=100


dict_keys(['meta', 'response'])

In [166]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    print(names.shape)
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
          results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
          print(f"not able to handle request for {name}")
          continue
          
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [167]:

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


(39,)
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
not able to handle request for Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfr

In [168]:
print(toronto_venues.shape)
toronto_venues.head()

(1478, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [169]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,...,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [170]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,...,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.010870,0.000000,0.000000,0.010870,0.021739,0.000000,0.000000,0.000000,0.010870,0.000000,0.010870,0.032609,0.000000,0.000000,0.000,0.010870,0.000000,0.032609,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.021739,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.054348,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010870,0.000000,0.043478,0.000000,0.010870,0.000000,0.000000,0.000,0.010870,0.000000,0.000000,0.010870,0.010870,0.010870,0.000000,0.000000,0.010870,0.00,0.010870,0.000,0.000000,0.010870,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.025641,0.000000,0.000000,0.025641,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.025641,0.000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.00,0.051282,0.025641,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,...,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.00,0.000000,0.000,0.000000,0.025641,0.000000,0.000000,0.000000,0.051282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.00,0.076923,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.033898,0.033898,0.050847,0.000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.016949,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.050847,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,...,0.000000,0.016949,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.016949,0.033898,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.00,0.000000,0.000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033898,0.00
4,Cent

In [171]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    display(temp.head())
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----


,index,0
0,Neighborhood,Berczy Park
1,Yoga Studio,0.0108696
2,Accessories Store,0
3,Afghan Restaurant,0
4,American Restaurant,0.0108696


                venue  freq
0         Coffee Shop  0.10
1                Café  0.05
2               Hotel  0.05
3  Seafood Restaurant  0.04
4          Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----


,index,1
0,Neighborhood,"Brockton, Parkdale Village, Exhibition Place"
1,Yoga Studio,0
2,Accessories Store,0.025641
3,Afghan Restaurant,0
4,American Restaurant,0


                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2               Gift Shop  0.05
3          Breakfast Spot  0.05
4  Thrift / Vintage Store  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----


...

                venue  freq
0    Greek Restaurant  0.22
1      Ice Cream Shop  0.05
2  Italian Restaurant  0.05
3                Café  0.05
4          Restaurant  0.05


----Toronto Dominion Centre, Design Exchange----


,index,36
0,Neighborhood,"Toronto Dominion Centre, Design Exchange"
1,Yoga Studio,0
2,Accessories Store,0
3,Afghan Restaurant,0
4,American Restaurant,0.03


                venue  freq
0               Hotel  0.09
1         Coffee Shop  0.09
2                Café  0.07
3          Restaurant  0.03
4  Seafood Restaurant  0.03


----University of Toronto, Harbord----


,index,37
0,Neighborhood,"University of Toronto, Harbord"
1,Yoga Studio,0.0333333
2,Accessories Store,0
3,Afghan Restaurant,0
4,American Restaurant,0


                 venue  freq
0                 Café  0.13
1   Italian Restaurant  0.07
2  Japanese Restaurant  0.07
3               Bakery  0.07
4            Bookstore  0.07




In [173]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Seafood Restaurant,Bakery,Cocktail Bar,Beer Bar,Japanese Restaurant,Restaurant,Deli / Bodega
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Breakfast Spot,Gift Shop,Thrift / Vintage Store,Grocery Store,Korean Restaurant,French Restaurant,Brewery,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Italian Restaurant,Sushi Restaurant,Bank,Bookstore,Deli / Bodega,Breakfast Spot,Martial Arts Dojo
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Bar,Italian Restaurant,Café,Bank,Gym / Fitness Center,Park,Wings Joint,French Restaurant,Bakery
4,Central Bay Street,Coffee Shop,Sandwich Place,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Clothing Store,Bubble Tea Shop,Department Store,Fast Food Restaurant


# 3.3 cluster neighborhoods by top venues

In [180]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

toronto_grouped_clustering.shape

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_grouped_clustering.head()
# add clustering labels
neighborhoods_venues_sorted.set_index('Neighborhood').head()

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# not all populated -Hui
toronto_merged = toronto_merged.dropna()

display(toronto_merged.head()) # check the last columns!


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0.0,Coffee Shop,Breakfast Spot,Health Food Store,Spa,Bakery,Beer Store,Distribution Center,Electronics Store,Event Space,Food Truck
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0.0,Sushi Restaurant,Coffee Shop,Gym,Distribution Center,Burrito Place,Creperie,Bubble Tea Shop,Restaurant,Martial Arts Dojo,Ramen Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0.0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Furniture / Home Store,Tea Room,Bubble Tea Shop,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0.0,Coffee Shop,Café,Seafood Restaurant,Gastropub,American Restaurant,Cocktail Bar,Italian Restaurant,Restaurant,Bakery,Creperie
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,0.0,Pub,Health Food Store,Trail,Miscellaneous Shop,Cheese Shop,Bakery,Gastropub,Deli / Bodega,Doner Restaurant,Farmers Market


# 3.4 Finally, display resulting clusters on map

In [181]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# 3.5 Examime clusters


In [185]:
clusters = ['Restaurant', 'studio', "park1", "park2", 'service']
for i in range(5):
  print(f"cluster label: {clusters[i]} ")
  display(toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]])
  

cluster label: Restaurant 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Coffee Shop,Breakfast Spot,Health Food Store,Spa,Bakery,Beer Store,Distribution Center,Electronics Store,Event Space,Food Truck
1,Downtown Toronto,0.0,Sushi Restaurant,Coffee Shop,Gym,Distribution Center,Burrito Place,Creperie,Bubble Tea Shop,Restaurant,Martial Arts Dojo,Ramen Restaurant
2,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Furniture / Home Store,Tea Room,Bubble Tea Shop,Pizza Place
3,Downtown Toronto,0.0,Coffee Shop,Café,Seafood Restaurant,Gastropub,American Restaurant,Cocktail Bar,Italian Restaurant,Restaurant,Bakery,Creperie
4,East Toronto,0.0,Pub,Health Food Store,Trail,Miscellaneous Shop,Cheese Shop,Bakery,Gastropub,Deli / Bodega,Doner Restaurant,Farmers Market
5,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Seafood Restaurant,Bakery,Cocktail Bar,Beer Bar,Japanese Restaurant,Restaurant,Deli / Bodega
6,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Clothing Store,Bubble Tea Shop,Department Store,Fast Food Restaurant
7,Downtown Toronto,0.0,Grocery Store,Café,Park,Playground,Candy Store,Baby Store,Coffee Shop,Distribution Center,Farmers Market,Event Space
8,Downtown Toronto,0.0,Coffee Shop,Café,Restaurant,Hotel,Gym,Salad Place,Japanese Restaurant,Steakhouse,Asian Restaurant,American Restaurant
9,West Toronto,0.0,Bakery,Park,Pet Store,Grocery Store,Furniture / Home Store,Middle Eastern Restaurant,Pharmacy,Pizza Place,Café,Bus Line


cluster label: studio 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1.0,Photography Studio,Park,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


cluster label: park1 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2.0,Park,Home Service,Trail,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store
23,Central Toronto,2.0,Gym Pool,Park,Playground,Garden,Discount Store,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
29,Central Toronto,2.0,Park,Gym,Grocery Store,Thai Restaurant,Women's Store,Discount Store,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store
33,Downtown Toronto,2.0,Park,Grocery Store,Playground,Candy Store,Women's Store,Discount Store,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store


cluster label: park2 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,3.0,Park,Women's Store,Distribution Center,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


cluster label: service 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4.0,Home Service,Women's Store,Distribution Center,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
